In [1]:
from amdex import AMdEX
import hashlib
import json

DATAEXCHANGE_URL = "https://develop.amdex.dev"
TRUST_URL = "https://develop.trust.amdex.dev"

email = "feduser-b@amdex.demo"
password = "yes"
password_hash = hashlib.sha256(password.encode()).hexdigest().upper()

In [2]:
# Create an instance of AMdEX class (it would make sense to call this trust instead of amdex)
amdex = AMdEX(TRUST_URL)
amdex.signin(email, password_hash)
trust_account_info = amdex.get_session()
print("trust_account_info:", trust_account_info)

trust_version_info = amdex.get_version()
print("trust_version_info:", trust_version_info)

trust_account_info: {'name': 'Predictive maintenance server', 'email': 'feduser-b@amdex.demo', 'account_uuid': 'cc633679-e4a6-4033-8b62-26ecc11ddb8e', 'auditlog_write_access': True, 'auditlog_read_access_from_consortium_uuids': [], 'auditlog_read_access_for_entries_by_account_uuids': ['6ae1ae4f-3c5c-4d55-8785-86076c6586f9', '886fc572-2522-4bf1-9399-3fca39fa946c', '35e825ef-739f-4c02-8e75-c8697021fc2e', 'cc633679-e4a6-4033-8b62-26ecc11ddb8e']}
trust_version_info: {'version': '0.0.3', 'api_version': '1.0.3', 'commit': '', 'dataexchange_url': 'https://develop.amdex.dev', 'trust_url': 'https://develop.trust.amdex.dev', 'federatedlearning_url': 'https://develop.federatedlearning.amdex.dev'}


In [3]:
dataexchange = AMdEX(DATAEXCHANGE_URL)
dataexchange.signin(email, password_hash)
dataexchange_account_info = dataexchange.get_session()
print("dataexchange_account_info:", dataexchange_account_info)

dataexchange_version_info = dataexchange.get_version()
print("dataexchange_version_info:", dataexchange_version_info)

dataexchange_account_info: {'name': 'Predictive maintenance server', 'email': 'feduser-b@amdex.demo', 'account_uuid': 'cc633679-e4a6-4033-8b62-26ecc11ddb8e', 'auditlog_write_access': True, 'auditlog_read_access_from_consortium_uuids': [], 'auditlog_read_access_for_entries_by_account_uuids': ['6ae1ae4f-3c5c-4d55-8785-86076c6586f9', '886fc572-2522-4bf1-9399-3fca39fa946c', '35e825ef-739f-4c02-8e75-c8697021fc2e', 'cc633679-e4a6-4033-8b62-26ecc11ddb8e']}
dataexchange_version_info: {'version': '0.0.3', 'api_version': '1.0.2', 'commit': '', 'dataexchange_url': 'https://develop.amdex.dev', 'trust_url': 'https://develop.trust.amdex.dev', 'federatedlearning_url': 'https://develop.federatedlearning.amdex.dev'}


In [4]:
# # Get dataviews
# dataviews = dataexchange.get_dataviews()
# print("Dataviews:", dataviews)

# # Create a dataview
# dataview_body = {
#     "name": "Sample Dataview",
#     "policy_uuids": ["example_policy_uuid1", "example_policy_uuid2"]
#     # "query": "SELECT * FROM table",
#     # ... other properties
# }
# new_dataview = dataexchange.create_dataview(dataview_body)
# print("New Dataview:", new_dataview)

# print("uuid", new_dataview)

# # Delete a dataview
# uuid = "731ea95e-d608-40db-91ae-fae98a8c0794"
# result = dataexchange.delete_dataview(new_dataview)
# print("Delete Result:", result)

In [5]:
#create an amdex class for the compliance officer
email_complianceofficer = "complianceofficer@amdex.demo"
password_complianceofficer = "yes"
password_hash_complianceofficer = hashlib.sha256(password_complianceofficer.encode()).hexdigest().upper()


# # Create an instance of AMdEX class
amdex_complianceofficer = AMdEX()
amdex_complianceofficer.signin(email_complianceofficer, password_hash_complianceofficer)

account_info_complianceofficer = amdex_complianceofficer.get_session()
print("account info compliance officer:", account_info_complianceofficer)



account info compliance officer: {'name': 'Compliance Officer', 'email': 'complianceofficer@amdex.demo', 'account_uuid': '9e8ed6aa-bfdb-4146-87d6-64ec538a258b', 'auditlog_write_access': False, 'auditlog_read_access_from_consortium_uuids': ['e9e92347-1077-4ef6-a21f-3cf40186f33a'], 'auditlog_read_access_for_entries_by_account_uuids': ['6ae1ae4f-3c5c-4d55-8785-86076c6586f9', '886fc572-2522-4bf1-9399-3fca39fa946c', '35e825ef-739f-4c02-8e75-c8697021fc2e', 'cc633679-e4a6-4033-8b62-26ecc11ddb8e']}


In [6]:
# Get policies
print("6. Get policies:")
policies = amdex_complianceofficer.get_policies()


print(f"Got {policies.__len__()} policies")
print("Policies:")
for policy in policies:
    print(f"Name: {policy['name']}\nUUID: {policy['policy_uuid']}\n")
print()

# Create policy
print("7. Create policy:")
policy_body = {
    "name": "Example policy 1",
    "inputs": [
        {
            "description": "Consortium UUID of data provider",
            "name": "provider_consortium_uuid",
            "source": "internal"
        },
        {
            "description": "Consortium UUID of data consumer",
            "name": "consumer_consortium_uuid",
            "source": "internal"
        }
    ],
    "outputs": [
        {
            "description": "Access allowed",
            "name": "access_allowed"
        }
    ],
    "url": {
        "human": "https://amdex.dev/policies/gdpr-attribution/README.txt",
        "legal": "https://amdex.dev/policies/gdpr-attribution/POLICY.txt",
        "machine": "https://amdex.dev/policies/gdpr-attribution/gdpr-attribution.eflint"
    },
    "hash_type": "SHA-256",
    "hash": {
        "human": "e5e61f918d66cd421ee506dff43f713b3e74449a23d035b65b8115196d7397d9",
        "legal": "d0119b1ff66c27f3973c0437bc2a19d426f6a5532313abca9cd9b7fdd37ba614",
        "machine": "8C2B4CA076E57619E239958200FF18D4CFC0E931755C2648D66B36F2BD4D7E0B"
    }
}

created_policy = amdex_complianceofficer.create_policy(policy_body)
print("Created Policy:", created_policy)
print()

# Delete policy
# print("8. Delete policy:")
# policy_uuid = created_policy.get("policy_uuid")
# deleted_policy = amdex_complianceofficer.delete_policy(policy_uuid)
# print("Deleted Policy:", deleted_policy)
# print()

6. Get policies:
Got 6 policies
Policies:
Name: Ex Post checks
UUID: 3a1c4ab9-3fe1-4b93-afc7-ebbd068afaa2

Name: Consortium partner allowed to access algorithm
UUID: 488c847c-b533-48db-931c-113e1ba35371

Name: Consortium partner allowed to access data
UUID: 21693473-d070-43bf-aca7-b75adbb51e48

Name: Data confirms to consortium rules
UUID: 8cceff93-ac99-4d10-a121-35413e3bea5a

Name: Consumer allowed to access provider data
UUID: cc0c5381-3c43-4c64-86d9-770655ebb696

Name: Example policy 1
UUID: 17858358-f8cf-4513-8b8f-7c92056c83a7


7. Create policy:
Created Policy: {'policy_uuid': 'fd2e3d0c-cf3b-4fc7-843e-cc31c4d4e0fc'}



In [7]:
# policies
print("6. Get policies:")
policies = amdex_complianceofficer.get_policies()
print("Policies:", policies)
print()

6. Get policies:
Policies: [{'name': 'Example policy 1', 'inputs': [{'description': 'Consortium UUID of data provider', 'name': 'provider_consortium_uuid', 'source': 'internal'}, {'description': 'Consortium UUID of data consumer', 'name': 'consumer_consortium_uuid', 'source': 'internal'}], 'outputs': [{'description': 'Access allowed', 'name': 'access_allowed'}], 'url': {'human': 'https://amdex.dev/policies/gdpr-attribution/README.txt', 'legal': 'https://amdex.dev/policies/gdpr-attribution/POLICY.txt', 'machine': 'https://amdex.dev/policies/gdpr-attribution/gdpr-attribution.eflint'}, 'hash_type': 'SHA-256', 'hash': {'human': 'E5E61F918D66CD421EE506DFF43F713B3E74449A23D035B65B8115196D7397D9', 'legal': 'D0119B1FF66C27F3973C0437BC2A19D426F6A5532313ABCA9CD9B7FDD37BA614', 'machine': '8C2B4CA076E57619E239958200FF18D4CFC0E931755C2648D66B36F2BD4D7E0B'}, 'policy_uuid': 'fd2e3d0c-cf3b-4fc7-843e-cc31c4d4e0fc', 'added': 1691481633955}, {'name': 'Ex Post checks', 'inputs': [{'description': 'Notary c

In [10]:
# Get jobs
print("9. Get jobs:")
jobs = amdex.get_jobs()
print("10. Existing Jobs:", jobs)

9. Get jobs:
10. Existing Jobs: [{'_id': '64d1f660515fa6d027f7f4d0', 'type': 'create', 'name': 'Jupyter Notebook Child Job', 'data': {'policy_uuids': ['fd2e3d0c-cf3b-4fc7-843e-cc31c4d4e0fc', 'fd2e3d0c-cf3b-4fc7-843e-cc31c4d4e0fc', 'fd2e3d0c-cf3b-4fc7-843e-cc31c4d4e0fc']}, 'context': {}, 'origin': 'https://develop.trust.amdex.dev', 'job_uuid': '50f87b3d-b9d3-4fc4-86a2-32c5671754a2', 'parent_job_uuid': 'baa39638-cace-48fa-abac-44bc4a6d3346', 'consumer_uuid': 'cc633679-e4a6-4033-8b62-26ecc11ddb8e', 'execution_plan': [{'type': 'Orchestrator', 'policy_uuid': 'fd2e3d0c-cf3b-4fc7-843e-cc31c4d4e0fc'}, {'type': 'Orchestrator', 'policy_uuid': 'fd2e3d0c-cf3b-4fc7-843e-cc31c4d4e0fc'}, {'type': 'Orchestrator', 'policy_uuid': 'fd2e3d0c-cf3b-4fc7-843e-cc31c4d4e0fc'}], 'started': 1691481696164, 'status': 'Executing plan item 1/3', 'execution_plan_index': 0, 'execution_plan_results': [], 'n_auditlog_entries': 3, 'latest_auditlog_timestamp': 1691481696365}, {'_id': '64d1f659515fa6d027f7f4cf', 'type': 'c

In [23]:
print("11. Create Parent job:")
parent_job_body = {
    "type": "create",
    "name": "Jupyter Notebook Parent Job",
    "data": { "policy_uuids": ["21693473-d070-43bf-aca7-b75adbb51e48", "8cceff93-ac99-4d10-a121-35413e3bea5a"] },
    "context": {
        "provider_consortium_uuid": "Parent job provider_consortium_uuid",
        "consumer_consortium_uuid": "Parent job consumer_consortium_uuid",
        "dataview_uuid": "Parent job dataview_uuid",
        "a_pip_url_from_context": DATAEXCHANGE_URL + "/api/pip"
    }
}
parent_job = amdex.create_job(parent_job_body)
parent_job_uuid = parent_job['job_uuid']
print("12. Parent Job:", parent_job)

print("13. Poll parent job status:")
parent_job_status = amdex.poll_job_status(parent_job_uuid)
print("14. Parent Job Status:", parent_job_status)
print()

11. Create Parent job:
{'type': 'create', 'name': 'Jupyter Notebook Parent Job', 'data': {'policy_uuids': ['21693473-d070-43bf-aca7-b75adbb51e48', '8cceff93-ac99-4d10-a121-35413e3bea5a']}, 'context': {'provider_consortium_uuid': 'Parent job provider_consortium_uuid', 'consumer_consortium_uuid': 'Parent job consumer_consortium_uuid', 'dataview_uuid': 'Parent job dataview_uuid', 'a_pip_url_from_context': 'https://develop.trust.amdex.dev/api/pip'}}
12. Parent Job: {'job_uuid': '911bc82c-d6c3-4c24-82e0-ac3254672fb6'}
13. Poll parent job status:
14. Parent Job Status: None



In [17]:
print("15. Create another job with our newly created policies and mark the previous job as parent:")
child_job_body = {
    "type": "create",
    "name": "Jupyter Notebook Child Job",
    "parent_job_uuid": parent_job_uuid,
    "data": { "policy_uuids": [created_policy['policy_uuid'], created_policy['policy_uuid'], created_policy['policy_uuid']] },
    "context": { 
        "provider_consortium_uuid": "Child job provider_consortium_uuid",
        "consumer_consortium_uuid": "Child job consumer_consortium_uuid"
    }
}
child_job = amdex.create_job(child_job_body)
child_job_uuid = child_job['job_uuid']

print("16. Child Job:", child_job)

print("17. Poll child job status:")
child_job_status = amdex.poll_job_status(child_job_uuid)

print("18. Child Job Status:", child_job_status)
print()

15. Create another job with our newly created policies and mark the previous job as parent:
{'type': 'create', 'name': 'Jupyter Notebook Child Job', 'parent_job_uuid': 'b84ab838-683b-49d1-b064-4b4bd60d3104', 'data': {'policy_uuids': ['fd2e3d0c-cf3b-4fc7-843e-cc31c4d4e0fc', 'fd2e3d0c-cf3b-4fc7-843e-cc31c4d4e0fc', 'fd2e3d0c-cf3b-4fc7-843e-cc31c4d4e0fc']}, 'context': {'provider_consortium_uuid': 'FederatedLearning child job provider_consortium_uuid', 'consumer_consortium_uuid': 'FederatedLearning child job consumer_consortium_uuid'}}
16. Child Job: {'job_uuid': '984c564c-e81b-45d2-b99e-566927a90ebc'}
17. Poll child job status:
18. Child Job Status: True



In [ ]:
# Delete all jobs
# print("18. Delete jobs:")
# deleted_jobs = amdex.delete_jobs()
# print("Deleted Jobs:", deleted_jobs)
# print()